In [8]:
import os
import time
import math
import random
import numpy as np
import pandas as pd
from google import genai
from google.genai import types
from dotenv import load_dotenv

In [9]:
random.seed(42)
load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
prompt_template = ''

In [3]:
def build_prompt(text):
    return prompt_template.replace('KALIMAT:', text)

In [4]:
def load_data():
    data = pd.read_csv('./val_data.csv', encoding='latin-1')
    return data

In [5]:
safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_CIVIC_INTEGRITY,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
]

def generate_noise(prompt):
    response = client.models.generate_content(
        model="gemini-2.5-flash-preview-04-17", 
        contents=prompt, 
        config=types.GenerateContentConfig(
            safety_settings=safety_settings,
        )
    )
    return response.text

# CHAR LEVEL NOISE

In [6]:
# Typos
prompt_template = f'''KALIMAT:

- lakukan noise injection dengan mensimulasikan typo pada kalimat tersebut.
- lakukan pada maksimal 25% dari kata yang ada pada kalimat tersebut.
- output hanya berupa kalimat akhir yang telah diubah
- jangan tambahkan markdown dan tanda baca apapun
- gunakan huruf kecil'''

In [7]:
data = load_data()
data.head()

,sentence,fuel,machine,others,part,price,service
0,fortuner kok boros banget ya tiap hari harus i...,negative,neutral,positive,neutral,neutral,neutral
1,servis di bengkel resmi Mitsubishi dengan laya...,neutral,neutral,neutral,neutral,neutral,positive
2,Toyota Rush ini gagah dan tetap nyaman dikenda...,neutral,neutral,positive,neutral,neutral,neutral
3,"tapi soal keamanan rush juara nya , karena dia...",positive,neutral,neutral,positive,neutral,neutral
4,"terus kalau sambil mengerem , HRV ini bisa kay...",neutral,neutral,neutral,negative,neutral,neutral


In [8]:
random.seed(42)
augmented_cnt = 0
for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['sentence']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["sentence"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)
        time.sleep(4)

Index: 1
Original: servis di bengkel resmi Mitsubishi dengan layanan Service Quick Pit . Cepat , tepat , dan mudah . 

Augmented: serfis di bengkel remsi mitsubishi dengan layanan service quick pit cepet tepat dan mudah
Index: 7
Original: @AstraHondaCare kecewa sama pelayanan bengkel resmi honda yang di balaraja , lebih tepat nya di depan gerbang permata balaraja , servis mahal dikira makin bagus eh malah kebalikan nya , 

Augmented: @astrahondacare keciwa sama pelayanann begkel resmi honda yang di balarjaa lebih tepat nya di depn gerbang permata balaraja servis mhal dikira makin bagus eh malah kebalikan nya
Index: 9
Original: servis nya payah masa mobil udah 3 bulan di bengkel belum bener-bener kecewa sama toyota 

Augmented: servis na payah masa mobul udah 3 bulan di bengkel belum bener-bener kecewa sam toyota
Index: 12
Original: Fortuner mahal tetapi minim fitur keamanan 

Augmented: fortuner mahal tetapi mnim fitur keamanan
Index: 19
Original: suka sama after sales service nya niss

In [9]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 25


In [10]:
data.to_csv('./noisy/typo.csv', index=False)

# SYNONYM REPLACEMENT

In [11]:
prompt_template = f'''KALIMAT:

- lakukan noise injection dengan mengganti maksimum 25% kata dalam kalimat dengan sinonim nya
- output hanya berupa 1 baris kalimat akhir yang telah diubah
- jangan tambahkan markdown dan tanda baca apapun
- jangan hilangkan kata selain dari kata yang diganti
- gunakan huruf kecil'''

In [12]:
data = load_data()
data.head()

,sentence,fuel,machine,others,part,price,service
0,fortuner kok boros banget ya tiap hari harus i...,negative,neutral,positive,neutral,neutral,neutral
1,servis di bengkel resmi Mitsubishi dengan laya...,neutral,neutral,neutral,neutral,neutral,positive
2,Toyota Rush ini gagah dan tetap nyaman dikenda...,neutral,neutral,positive,neutral,neutral,neutral
3,"tapi soal keamanan rush juara nya , karena dia...",positive,neutral,neutral,positive,neutral,neutral
4,"terus kalau sambil mengerem , HRV ini bisa kay...",neutral,neutral,neutral,negative,neutral,neutral


In [14]:
random.seed(42)
augmented_cnt = 0
for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['sentence']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["sentence"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)
        time.sleep(4)

Index: 1
Original: servis di bengkel resmi Mitsubishi dengan layanan Service Quick Pit . Cepat , tepat , dan mudah . 

Augmented: perawatan di bengkel resmi Mitsubishi dengan jasa service quick pit . lekas , tepat , dan mudah .
Index: 7
Original: @AstraHondaCare kecewa sama pelayanan bengkel resmi honda yang di balaraja , lebih tepat nya di depan gerbang permata balaraja , servis mahal dikira makin bagus eh malah kebalikan nya , 

Augmented: @astrahondacare jengkel sama layanan bengkel sah honda yang di balaraja lebih pas nya di depan gerbang permata balaraja servis premium disangka semakin bagus eh malah kebalikan nya
Index: 9
Original: servis nya payah masa mobil udah 3 bulan di bengkel belum bener-bener kecewa sama toyota 

Augmented: servis nya buruk masa kendaraan sudah 3 bulan di bengkel belum sungguh-sungguh jengkel sama toyota
Index: 12
Original: Fortuner mahal tetapi minim fitur keamanan 

Augmented: fortuner mahal tetapi kurang fitur keamanan
Index: 19
Original: suka sama aft

In [15]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 25


In [16]:
data.to_csv('./noisy/synonym-replacement.csv', index=False)

# WORD INSERTION

In [20]:
prompt_template = f'''KALIMAT:

- lakukan noise injection dengan menyisipkan kata-kata irrelevan yang membuat kalimat menjadi tidak nyambung
- sebar 3-6 kata tambahan di seluruh kalimat
- jangan mengubah kata yang sudah ada, hanya sisipkan kata baru
- output hanya berupa 1 baris kalimat akhir yang telah diubah
- jangan tambahkan markdown dan tanda baca apapun
- jangan hilangkan kata selain dari kata yang diganti
- gunakan huruf kecil'''

In [21]:
data = load_data()
data.head()

,sentence,fuel,machine,others,part,price,service
0,fortuner kok boros banget ya tiap hari harus i...,negative,neutral,positive,neutral,neutral,neutral
1,servis di bengkel resmi Mitsubishi dengan laya...,neutral,neutral,neutral,neutral,neutral,positive
2,Toyota Rush ini gagah dan tetap nyaman dikenda...,neutral,neutral,positive,neutral,neutral,neutral
3,"tapi soal keamanan rush juara nya , karena dia...",positive,neutral,neutral,positive,neutral,neutral
4,"terus kalau sambil mengerem , HRV ini bisa kay...",neutral,neutral,neutral,negative,neutral,neutral


In [22]:
random.seed(42)
augmented_cnt = 0
for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['sentence']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["sentence"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)

        time.sleep(4)

Index: 1
Original: servis di bengkel resmi Mitsubishi dengan layanan Service Quick Pit . Cepat , tepat , dan mudah . 

Augmented: servis apel di awan bengkel resmi sepatu Mitsubishi dengan senyum layanan Service kucing Quick Pit Cepat buku tepat dan mudah
Index: 7
Original: @AstraHondaCare kecewa sama pelayanan bengkel resmi honda yang di balaraja , lebih tepat nya di depan gerbang permata balaraja , servis mahal dikira makin bagus eh malah kebalikan nya , 

Augmented: kecewa kucing sama biru pelayanan nasi bengkel pelan resmi dan honda makan yang di balaraja lebih tepat nya di depan gerbang permata balaraja servis mahal dikira makin bagus eh malah kebalikan nya
Index: 9
Original: servis nya payah masa mobil udah 3 bulan di bengkel belum bener-bener kecewa sama toyota 

Augmented: servis nya pisang payah masa mobil biru udah 3 bulan di bengkel belum bener-bener jendela kecewa sama terbang toyota
Index: 12
Original: Fortuner mahal tetapi minim fitur keamanan 

Augmented: fortuner kucing

In [23]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 25


In [24]:
data.to_csv('./noisy/word-insertion.csv', index=False)

# WORD DELETION

In [14]:
data = load_data()
data.head()

,sentence,fuel,machine,others,part,price,service
0,fortuner kok boros banget ya tiap hari harus i...,negative,neutral,positive,neutral,neutral,neutral
1,servis di bengkel resmi Mitsubishi dengan laya...,neutral,neutral,neutral,neutral,neutral,positive
2,Toyota Rush ini gagah dan tetap nyaman dikenda...,neutral,neutral,positive,neutral,neutral,neutral
3,"tapi soal keamanan rush juara nya , karena dia...",positive,neutral,neutral,positive,neutral,neutral
4,"terus kalau sambil mengerem , HRV ini bisa kay...",neutral,neutral,neutral,negative,neutral,neutral


In [15]:
random.seed(42)
augmented_cnt = 0
for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['sentence']
        words = text.split()
        num_words = len(words)
        num_to_del = math.ceil(len(text.split(' ')) / 4)

        print("Index:", i)
        print("Original:", text, '\n')

        if num_to_del > 0:
            indices_to_delete = np.random.choice(num_words, size=num_to_del, replace=False)
            mask = np.ones(num_words, dtype=bool)
            mask[indices_to_delete] = False
            augmented_words = np.array(words)[mask]
            augmented = ' '.join(augmented_words)
        else:
            augmented = text  # if too short, don't change

        data.at[i, "sentence"] = augmented

        print("Augmented:", augmented)
        print("=" * 50)
        # time.sleep(5)

Index: 1
Original: servis di bengkel resmi Mitsubishi dengan layanan Service Quick Pit . Cepat , tepat , dan mudah . 

Augmented: servis di bengkel resmi layanan Service Quick Pit . , tepat dan .
Index: 7
Original: @AstraHondaCare kecewa sama pelayanan bengkel resmi honda yang di balaraja , lebih tepat nya di depan gerbang permata balaraja , servis mahal dikira makin bagus eh malah kebalikan nya , 

Augmented: @AstraHondaCare kecewa sama pelayanan resmi honda yang , tepat nya di depan gerbang balaraja , mahal dikira makin bagus kebalikan nya ,
Index: 9
Original: servis nya payah masa mobil udah 3 bulan di bengkel belum bener-bener kecewa sama toyota 

Augmented: servis nya payah masa udah 3 bulan di bengkel belum kecewa
Index: 12
Original: Fortuner mahal tetapi minim fitur keamanan 

Augmented: Fortuner minim fitur keamanan
Index: 19
Original: suka sama after sales service nya nissan yang murah 

Augmented: suka after service nya nissan yang
Index: 26
Original: Punya Kakak saya HRV - n

In [16]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 25


In [17]:
data.to_csv('./noisy/word-deletion.csv', index=False)

# COMBINED

In [6]:
prompt_template = f'''KALIMAT:

- lakukan 3 noise injection: typo, penghapusan kata, atau penyisipan kata-kata irrelevan yang membuat kalimat menjadi tidak nyambung
- lakukan noise injection pada maksimum 40% kata dari kalimat
- pastikan kalimat akhir memiliki masing-masing macam noise injection
- jika ada beberapa kata yang sama, hanya ubah satu kata saja
- jangan tambahkan markdown dan tanda baca apapun
- gunakan huruf kecil
- output hanya berupa kalimat akhir yang telah diubah
'''

In [7]:
data = load_data()
data.head()

,sentence,fuel,machine,others,part,price,service
0,fortuner kok boros banget ya tiap hari harus i...,negative,neutral,positive,neutral,neutral,neutral
1,servis di bengkel resmi Mitsubishi dengan laya...,neutral,neutral,neutral,neutral,neutral,positive
2,Toyota Rush ini gagah dan tetap nyaman dikenda...,neutral,neutral,positive,neutral,neutral,neutral
3,"tapi soal keamanan rush juara nya , karena dia...",positive,neutral,neutral,positive,neutral,neutral
4,"terus kalau sambil mengerem , HRV ini bisa kay...",neutral,neutral,neutral,negative,neutral,neutral


In [8]:
random.seed(42)
augmented_cnt = 0

for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['sentence']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["sentence"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)

        time.sleep(5)

Index: 1
Original: servis di bengkel resmi Mitsubishi dengan layanan Service Quick Pit . Cepat , tepat , dan mudah . 

Augmented: servia di bengkel resmi Mitsubishi pisang layanan service quick pit cepat tepat dan mudah
Index: 7
Original: @AstraHondaCare kecewa sama pelayanan bengkel resmi honda yang di balaraja , lebih tepat nya di depan gerbang permata balaraja , servis mahal dikira makin bagus eh malah kebalikan nya , 

Augmented: @astrahondacare kecewa sama pisang pelayanan bengkel honda yang di balarja lebih tepat di depan langit gerbang permata balaraja servis mahal dikira makin bagusss eh malah kebalikann nya
Index: 9
Original: servis nya payah masa mobil udah 3 bulan di bengkel belum bener-bener kecewa sama toyota 

Augmented: serpis nya masa mobil udah 3 bulan di bengkel belum kucing bener-bener kecewa sama toyota
Index: 12
Original: Fortuner mahal tetapi minim fitur keamanan 

Augmented: fortuner mahall dingin minim fitur keamanan
Index: 19
Original: suka sama after sales ser

In [9]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 25


In [10]:
data.to_csv('./noisy/combined-noise.csv', index=False)